# https://www.youtube.com/watch?v=P8tOjiYEFqU

In [5]:
import pandas as pd
from openai import OpenAI
import os
import ast
import numpy as np
import pdb

In [6]:
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
question = "What temperature do i set my house on Vacation so the pipes don't freeze"
question

In [4]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    message=[
        {
            "role":"user",
            "content":question
        }
    ]
)
response

NameError: name 'client' is not defined

In [ ]:
response.choices[0].message.content

# Now we will add Retrieval Augmented Generation

# first we create a little vectorDB from all the webpages

In [ ]:
question = "Do you have parking"
question

In [ ]:
df = pd.read_csv('webpage-text.csv')

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n"," ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
get_embedding(df['text'].iloc[0]

In [ ]:
%%time
df['text'].head(5).apply(get_embedding)

In [ ]:
%%time

In [ ]:
%%time
df['embeddings'] = df['text'].apply(get_embedding)
df.to_csv('./website-with-embeddings.csv',index=False)
df.to_picke('./website-with-embeddings.pkl')

In [ ]:
%%time
df.pd.read_pickle('./website-with-embeddings.pkl')
df.head()

In [ ]:
question_embedding = get_embedding(question)
question, question_embedding[0:10], "..."

In [ ]:
def distance_calculations(page_embedding):
        return np.dot(page_embedding, question_embedding)
df['distance'] = df['embeddings'].apply(distance_calculations)
df.head()

In [ ]:
context = df['text'].iloc[0] + "\n" + df['text'].iloc[2] + "\n" + df['text'].iloc[4] + "\n"
print(context)

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistante who is helping Khaata.in, a Saas product respond to seller questions. When you answer the question, use the first person (.e.g We) to refer to Khaata."},
        {"role":"user","content":question),
        {"role", "assistant", "content": f"Use this information from Khaata.in's website as context to answer the user question: {context}. Please stick to this context when answering the question.")
])

In [ ]:
response.choices[0].message.content

## Now making it repeatable

In [ ]:
def query(question): 
    question_embedding = get_embedding(question)

    def distance_calculations(page_embedding):
        return np.dot(page_embedding, question_embedding)
        
    distance_series = df['embeddings'].apply(distance_calculations)
    top_four = distance_series.sort_values(ascending=False).index[0:4]

    text_series = df.loc[top_four].['text']
    context = "\n\n".join(text_series)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistante who is helping Khaata.in, a Saas product respond to seller questions. When you answer the question, use the first person (.e.g We) to refer to Khaata."},
            {"role":"user","content":question),
            {"role", "assistant", "content": f"Use this information from Khaata.in's website as context to answer the user question: {context}. Please stick to this context when answering the question.")
    ])
    
    response.choices[0].message.content

In [ ]:
query("do you have parking")

In [ ]:
query("do you accept Medicare")